# Jet Energy Resolution Computation

In [ ]:
import awkward as ak
import numpy as np
import coffea
import uproot
import hist
import vector
from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from distributed.diagnostics.plugin import UploadDirectory
import matplotlib.pyplot as plt
from collections import defaultdict
import os
import pickle
import correctionlib
from coffea.analysis_tools import PackedSelection
from dask.distributed import Client
from scipy.optimize import curve_fit
import csv
np.seterr(divide='ignore', invalid='ignore')

In [ ]:
with open("pkl_files/QCD_pt_response_NEW.pkl", "rb") as f:
    output = pickle.load(f)
    
for k,v in output.items():
    print(k)

In [ ]:
output['cutflow'].keys() 

### Loading and Plotting PKL File Output

Let's start by plotting the histograms that contain data for the ratio of reconstructed jet $p_{T}$ to generated jet $p_{T}$; we use the `output["responses_histogram"]` key and project it to `dataset` and the variable of interest, such as `.project("dataset", "pt")`, `.project("dataset", "frac")`, or `.project("dataset", "eta")`, for example.

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 5))

output["responses_histogram"].project('dataset', 'frac').plot(ax=axs[0], density=True)
output["responses_histogram"].project('dataset', 'eta').plot(ax=axs[1], density=True)
output["responses_histogram"].project('dataset', 'pt').plot(ax=axs[2], density=True)
                                                                  
axs[0].legend(frameon=True)
axs[1].legend(frameon=True)
axs[2].legend(frameon=True)
axs[2].set_yscale('log')
axs[2].set_xscale('log')
plt.show()

In [ ]:
print(f".project('dataset', 'frac') numpy shape    ==>    {output['responses_histogram'].project('dataset', 'frac').to_numpy()[0].shape}")
print(f".project('dataset', 'eta') numpy shape    ==>    {output['responses_histogram'].project('dataset', 'eta').to_numpy()[0].shape}")
print(f".project('dataset', 'pt') numpy shape    ==>    {output['responses_histogram'].project('dataset', 'pt').to_numpy()[0].shape}")

Next, we project the `output["corrections_histogram"]` histogram to its corresponding axes as well:

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 5))

output["corrections_histogram"].project('dataset', 'rho').plot(ax=axs[0], density=True)
output["corrections_histogram"].project('dataset', 'npvs').plot(ax=axs[1], density=True)
output["corrections_histogram"].project('dataset', 'npu').plot(ax=axs[2], density=True)

axs[0].legend(frameon=True)
axs[1].legend(frameon=True)
axs[2].legend(frameon=True)
plt.show()

In [ ]:
print(f".project('dataset', 'rho') numpy shape    ==>    {output['corrections_histogram'].project('dataset', 'rho').to_numpy()[0].shape}")
print(f".project('dataset', 'npvs') numpy shape    ==>    {output['corrections_histogram'].project('dataset', 'npvs').to_numpy()[0].shape}")
print(f".project('dataset', 'npu') numpy shape    ==>    {output['corrections_histogram'].project('dataset', 'npu').to_numpy()[0].shape}")

### Fitting Histograms and Plotting Curves for a Given Transverse Momentum Range

In order to add a fit to our historgrams, we start by defining the edges, centers, and widths of the bins for our quantities of interest, which will be the `dataset`, `frac`,`eta`, and `pt`. We also define the define a dataset axis for future use:

In [ ]:
histogram_1 = output["responses_histogram"]

for axis in histogram_1.axes:
    print(f"Axis type: {axis}\n")
    
dataset_axis = histogram_1.axes[0]

In [ ]:
bin_edges = []
bin_centers = []
bin_widths = []

eras = []

for i in range(len(dataset_axis)):
    era = dataset_axis[i]
    eras.append(era)

print(eras)

for axis in histogram_1.axes:
    bin_edges.append(axis.edges)
    bin_centers.append(axis.centers)
    bin_widths.append(axis.widths)

dataset_bin_edges, frac_bin_edges, eta_bin_edges, pt_bin_edges = bin_edges

dataset_bin_centers, frac_bin_centers, eta_bin_centers, pt_bin_centers = bin_centers

dataset_bin_widths, frac_bin_widths, eta_bin_widths, pt_bin_widths = bin_widths

In [ ]:
print(f"pt_bin_edges: {pt_bin_edges}\n\npt_bin_centers: {pt_bin_centers}\n\npt_bin_widths: {pt_bin_widths}\n")

Having defined our bin dimensions, we now fit our histograms to a gaussian function and print its relevant parameters, such as its amplitude, mean, and width. It is important to note that we are projecting the $p_{T}$ fraction ratio over the entire dataset through `.project('dataset', 'frac')`, and so the distribution obtained contains the response over its whole respective era. Our next goal will be to create a fit for each $p_{T}$ bin, for each complete era, as this will allow us to obtain the widths (JER) and means (JES) of the graphs as functions of the transverse momentum bins for all four eras. To do so, we will first need to convert our `Hist` objects to numpy arrays.

In [ ]:
def gaussian_function(x, amplitude, mean, standard_dev):
    return amplitude * np.exp(- (x - mean)**2 / (2. * standard_dev**2))

histogram_frac_numpy, bin_edges = histogram_1.project('dataset', 'frac').to_numpy()[0], frac_bin_edges
bin_centers = frac_bin_centers
frac_xspace = np.linspace(0, 2, 300)

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

for i in range(len(eras)):
    normalized_y_vals = histogram_frac_numpy[i] / np.sum(histogram_frac_numpy[i])
    popt, pcov = curve_fit(gaussian_function, xdata=bin_centers, ydata=normalized_y_vals)
    print(f"Era: {eras[i]}    |    amplitude: {popt[0]}    |    mean: {popt[1]}    |    stdev: {popt[2]}    |    .project('dataset', 'frac'): {output['responses_histogram'].project('dataset', 'frac').to_numpy()[0].shape}\n")
    
    axs[0].stairs(normalized_y_vals, bin_edges, label=eras[i])
    axs[0].set_title("$p_T$ Response, Experiment")
    axs[0].set_xlabel("Response")
    axs[0].set_ylabel("Normalized Counts")
    axs[0].legend(frameon=True)
    
    axs[1].plot(frac_xspace, gaussian_function(frac_xspace, *popt), label=eras[i])
    axs[1].set_title("$p_T$ Response, Gaussian Fits")
    axs[1].set_xlabel("Response")
    axs[1].set_ylabel("Normalized Counts")
    axs[1].legend(frameon=True)

Our last task will be to record the means and widths of the gaussian fits for all $p_{T}$ bins, and then plot them as functions of the transverse momentum. We will then fit these JES/JER curves to the text files in the `jer_files` folder.

### Obtaining JES and JER Datapoints

Before continuing further, lets plot first all transverse momentum bins for each era; this will allow us to see $p_{T}$ distribution more clearly. To start with, we have to define a new histogram projection `.project('dataset', 'pt')` that contains all $p_{T}$ bins:

In [ ]:
histogram_2 = output["responses_histogram"]
histogram_pt_numpy, bin_edges = histogram_2.project("dataset", "pt").to_numpy()[0], pt_bin_edges
bin_widths = pt_bin_widths

for i in range(len(eras)):
    normalized_y_vals = histogram_pt_numpy[i] / (np.sum(histogram_pt_numpy[i]) * bin_widths)
    plt.stairs(normalized_y_vals, bin_edges, label=eras[i])
    plt.title("$p_T$ Distributions")
    plt.xlabel(r"$p_T$ [GeV]")
    plt.xscale("log")
    plt.legend(frameon=True)

print(f".project('dataset', 'pt') numpy shape    ==>    {histogram_2.project('dataset', 'pt').to_numpy()[0].shape}\n")

Now we define the projection `.project('pt', 'frac')` and plot the fractional transverse momentum response for all eras, for each $p_{T}$ bin:

In [ ]:
histogram_3 = output["responses_histogram"]
histogram_pt_frac_numpy, bin_edges = histogram_3.project("pt", "frac").to_numpy()[0], frac_bin_edges

color_map = plt.get_cmap('brg', len(pt_bin_centers))
    
for i in range(len(pt_bin_centers)):
    color = color_map(i / len(pt_bin_centers))
    normalized_y_vals = histogram_pt_frac_numpy[i] / np.sum(histogram_pt_frac_numpy[i])
    plt.stairs(normalized_y_vals, bin_edges, label=f"${pt_bin_edges[i]} <  p_{{T}}  < {pt_bin_edges[i + 1]}$", color=color)
    plt.title("$p_T$ Response, All Eras")
    plt.xlabel("Response")
    plt.ylabel("Normalized Counts")
    #plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=3)
    
print(f".project('pt', 'frac') numpy shape    ==>    {histogram_3.project('pt', 'frac').to_numpy()[0].shape}\n")

If we now define one last projection `.project('dataset', 'pt', 'frac')`, we can plot the $p_{T}$ response curves separately for each individual dataset:

In [ ]:
histogram_4 = output["responses_histogram"]
histogram_dataset_pt_frac_numpy, bin_edges = histogram_4.project("dataset", "pt", "frac").to_numpy()[0], frac_bin_edges

color_map = plt.get_cmap('brg', len(pt_bin_centers))

for i, dataset in enumerate(histogram_dataset_pt_frac_numpy):
    for j, frac_vals in enumerate(dataset):
        color = color_map(j, len(pt_bin_centers))
        normalized_y_vals = frac_vals / np.sum(frac_vals)
        plt.stairs(normalized_y_vals, bin_edges, label=f"${pt_bin_edges[j]} <  p_{{T}}  < {pt_bin_edges[j + 1]}$", color=color)
        plt.title(f"$p_T$ Response, {histogram_4.project('dataset', 'pt', 'frac').axes[0][i]}")
        plt.xlabel("Response")
        plt.ylabel("Normalized Counts")
    #plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=3)
    plt.show()
    
print(f".project('dataset', 'pt', 'frac') numpy shape    ==>    {histogram_4.project('dataset', 'pt', 'frac').to_numpy()[0].shape}\n")

Finally, we fit all response curves of each era to a gaussian, and save their amplitudes, means, widths, and respective uncertainties to a CSV file:

In [ ]:
histogram_4 = output["responses_histogram"]
histogram_dataset_pt_frac_numpy, bin_edges = histogram_4.project("dataset", "pt", "frac").to_numpy()[0], frac_bin_edges
bin_centers = frac_bin_centers
frac_xspace = np.linspace(0, 2, 300)

fig, axs = plt.subplots(4, 2, figsize=(20, 20))

fit_parameters = {}

for i, dataset in enumerate(histogram_dataset_pt_frac_numpy):
    era_parameters = []
    for j, frac_vals in enumerate(dataset):
        normalized_y_vals = frac_vals / np.sum(frac_vals)
        popt, pcov = curve_fit(gaussian_function, xdata=bin_centers, ydata=np.nan_to_num(normalized_y_vals))
        era_parameters.append((popt, pcov))
            
        axs[i, 0].stairs(normalized_y_vals, bin_edges)
        axs[i, 0].set_title(f"$p_T$ Response {histogram_4.project('dataset', 'pt', 'frac').axes[0][i]}, Experiment")
        axs[i, 0].set_xlabel("Response")
        axs[i, 0].set_ylabel("Normalized Counts")
    
        axs[i, 1].plot(frac_xspace, gaussian_function(frac_xspace, *popt))
        axs[i, 1].set_title(f"$p_T$ Response {histogram_4.project('dataset', 'pt', 'frac').axes[0][i]}, Gaussian Fits")
        axs[i, 1].set_xlabel("Response")
        axs[i, 1].set_ylabel("Normalized Counts")
    fit_parameters[histogram_4.project('dataset', 'pt', 'frac').axes[0][i]] = era_parameters

In [ ]:
output_dir = "./gaussian_fit_files/"
os.makedirs(output_dir, exist_ok=True)

for key, value in fit_parameters.items():
    filename = f"gaussian_parameters_{key}.csv"
    filepath = os.path.join(output_dir, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['popt', 'pcov'])       
        for popt, pcov in value:
            writer.writerow([popt, pcov])